In [ ]:
from pathlib import Path

import numpy as np
import xarray as xr
from siphon.catalog import TDSCatalog

from atm_forcing import CF_ROMS, generate_catalog_urls, get_ds, reshape_to_full_year

### Download, preprocess, save 

In [ ]:
LAT_NEW = np.arange(58.9, 60, 0.02)
LON_NEW = np.arange(10.1, 11.1, 0.02)

In [ ]:
file_path_grid = Path.home() / "dump_fram_nn9297k" / "ROHO800_grid_fix5.nc"
ds_grid = xr.open_dataset(file_path_grid)

In [ ]:
ds_grid

In [ ]:
regridder = None
dss = []
timestamps = []
parameters = [x[0] for x in CF_ROMS]
for date_and_time, catalog_url in generate_catalog_urls(start_year=2005, end_year=2021):
    timestamp = date_and_time.strftime("%Y%m%d")
    file_path = Path.home() / "NORA3" / f"{timestamp}.nc"
    if file_path.exists():
        print(f"File {file_path} exists for {date_and_time}.")
        continue
    print(f"Processing: {date_and_time}.")
    cat = TDSCatalog(catalog_url)
    urls = [v.access_urls["opendap"] for k, v in cat.datasets.items() if "_fp" in k]
    ds = xr.open_mfdataset(urls, combine="by_coords", compat="no_conflicts", data_vars="all")
    # ds = ds[parameters]
    break
    regridder, ds_out = get_ds(regridder, ds, LAT_NEW, LON_NEW)
    dss.append(ds)
    timestamps.append(timestamp)
    if len(dss) > 3:  # there should be 4 files per day.
        assert len(set(timestamps)) <= 1
        ds = xr.combine_by_coords(dss, coords=["time"], join="outer")
        print(f"Saving to {file_path}.")
        ds.to_netcdf(file_path)  # , encoding={var: {"zlib": True, "complevel": 5} for var in ds.data_vars})
        print("Saving done.")
        dss = []
        timestamps = []

In [ ]:
def lonlat_to_angle(lon, lat):
    # this returns angle from east to the current x
    diff_lon = np.diff(lon, axis=1)
    diff_lon = np.hstack([diff_lon, diff_lon[:, -1:]])
    diff_lat = np.diff(lat, axis=1)
    diff_lat = np.hstack([diff_lat, diff_lat[:, -1:]])
    assert np.all(np.abs(diff_lon) < 180)
    diff_lon *= np.cos(np.deg2rad(lat))
    return np.arctan2(diff_lat, diff_lon)

In [ ]:
angle = lonlat_to_angle(ds.longitude.values, ds.latitude.values)

we should rotate in the opposite direction

In [ ]:
def rotate_u_v(angle, u_east, v_north):
    # rotate in the direction of angle
    cos_alpha = np.cos(angle)
    sin_alpha = np.sin(angle)
    u_x = u_east * cos_alpha + v_north * sin_alpha
    v_y = v_north * cos_alpha - u_east * sin_alpha
    return u_x, v_y

In [ ]:
u, v = rotate_u_v(-angle, ds.x_wind_10m, ds.y_wind_10m)

In [ ]:
da_u = xr.DataArray(
    data=u,
    coords=ds.x_wind_10m.coords,
    dims=ds.x_wind_10m.dims,
    name="u"
)

In [ ]:
ds.x_wind_10m.isel(time=0, height4=0).plot()

In [ ]:
def wind_direction_transform(da):
    # blows from to blows to
    da += 180
    da = xr.where(da >= 360, da - 360, da)
    # clockwise to unticlockwise
    da = -1 * da + 360
    # rotate so east is pos x and north is pos y
    da += 90
    da = xr.where(da >= 360, da - 360, da)
    return (np.pi / 180) * da

In [ ]:
da_wd = wind_direction_transform(ds.wind_direction.copy(deep=True))

In [ ]:
u = ds.wind_speed * np.cos(da_wd)
v = ds.wind_speed * np.sin(da_wd)

In [ ]:
da_u.isel(time=0, height4=0).plot()

In [ ]:
u.isel(time=0, height4=0).plot()

### Checkup

In [ ]:
pattern = str(Path.home() / "FjordSim_data" / "NORA3" / "[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9].nc")
ds = xr.open_mfdataset(pattern, combine="by_coords")

In [ ]:
ds = reshape_to_full_year(ds)

In [ ]:
bad_steps = ds.time.diff("time") != np.timedelta64(1, "h")
print("Bad steps at positions:", bad_steps.where(bad_steps, drop=True))

In [ ]:
ds

In [ ]:
ds.to_netcdf(
    Path.home() / "FjordSim_data" / "NORA3" / "NORA3.nc",
    encoding={var: {"zlib": True, "complevel": 5} for var in ds.data_vars},
)

### Saving atm parameters separately

In [ ]:
for var_name, da in ds.data_vars.items():
    da = da.rename(var_name)
    filename = f"{var_name}.nc"
    da.to_netcdf(
        Path.home() / "FjordSim_data" / "NORA3" / filename, encoding={var_name: {"zlib": True, "complevel": 5}}
    )
    print(f"{filename} saved.")